In [1]:
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from config import password
import pandas as pd

In [2]:
breweries= pd.read_csv('data/breweries.csv')
popular= pd.read_csv('data/beer_mostpopular_clean.csv')
rated= pd.read_csv('data/beer_toprated_clean.csv')

In [3]:
breweries = breweries.set_index('id',drop=True)
breweries.head(10)

,name,street_address,zipcode,austin_area,untappd_website
id,,,,,
0,4th Tap Brewing Cooperative,10615 Metric Boulevard,78758,North,https://untappd.com/4thTap/beer
1,Adelbert’s Brewery,2314 Rutland Drive,78758,North,https://untappd.com/adelbertsbrewery/beer
2,Austin Beerworks,3001 Industrial Terrace,78758,North,https://untappd.com/austinbeerworks/beer
3,Black Star Co-Op,7020 Easy Wind Drive,78752,North,https://untappd.com/BlackStarCo-op/beer
4,Brewtorium,6015 Dillard Circle,78752,North,https://untappd.com/Brewtorium/beer
5,Celis Brewing,10001 Metric Boulevard,78758,North,https://untappd.com/CelisBrewery/beer
6,Circle Brewing,2340 W Braker Lane,78758,North,https://untappd.com/circlebrewing/beer
7,Draught House,4112 Medical Parkway,78756,North,https://untappd.com/Draught_House/beer
8,Hopsquad Brewing,2307 Kramer Lane,78758,North,https://untappd.com/w/hopsquad-brewing-co/4376...


In [4]:
popular = popular.reset_index()
popular = popular.rename(columns = {'index': 'id'})
popular = popular.set_index('id',drop=True)
popular.head(10)

,brewery_id,name,type,abv,ibu,rating,rating_count
id,,,,,,,
0,0,Long Walk,IPA - American,7.0,57.0,3.61,5722
1,0,Kung Fu Robot,IPA - American,7.0,43.0,3.65,4870
2,0,Vitality Light Ale,Blonde Ale,5.0,17.0,3.46,2725
3,0,Sun Eater,Gruit / Ancient Herbed Ale,4.8,NaN,3.49,2588
4,0,Highland Scoundrel,Scotch Ale / Wee Heavy,10.6,20.0,3.84,2420
5,1,Naked Nun,Wheat Beer - Witbier,5.8,NaN,3.60,10949
6,1,Tripel B,Belgian Tripel,9.3,NaN,3.87,10281
7,1,Philosophizer,Farmhouse Ale - Saison,7.8,36.0,3.80,9766
8,1,Hibiscus Saison,Farmhouse Ale - Saison,5.6,33.0,3.65,7332


In [5]:
rated = rated.reset_index()
rated = rated.rename(columns = {'index': 'id'})
rated = rated.set_index('id',drop=True)
rated.head(10)

,brewery_id,name,type,abv,ibu,rating,rating_count
id,,,,,,,
0,0,Barrel Aged Lawgiver,Stout - Russian Imperial,10.0,NaN,4.31,36
1,0,Barrel Aged Supernaut (Garrison Brothers),Stout - Imperial / Double,13.0,NaN,4.29,17
2,0,Fog of War,IPA - New England,6.8,40.0,4.18,11
3,0,Supernaut Cookies & Cream (Barrel Aged Garriso...,Stout - Imperial / Double,13.0,NaN,4.10,60
4,0,Cask Collective Blood Orange,IPA - American,7.0,NaN,4.10,10
5,1,Guava Sour,Sour - Other,NaN,NaN,4.15,137
6,1,Berrial Shroud,Sour - Fruited,6.5,NaN,4.15,104
7,1,Forgotten Ale,Sour - Flanders Oud Bruin,9.0,NaN,4.13,168
8,1,Sundowner (Biere Brut) (2018 -),Bière de Champagne / Bière Brut,12.1,NaN,4.11,16


In [ ]:
rds_connection_string = f"postgres:{password}@localhost:5432/austin_breweries"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [ ]:
session = Session(engine)
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

# Save reference to the table
Base.classes.keys()

In [ ]:
breweries.to_sql(name='breweries', con=engine, if_exists='append', index=True)


In [ ]:
popular.to_sql(name='popular_beers', con=engine, if_exists='replace', index=True)

In [ ]:
rated.to_sql(name='top_rated_beers', con=engine, if_exists='replace', index=True)

In [6]:
engine2 = create_engine("sqlite:///data/austin_breweries.sqlite")
con = engine2.connect()

In [14]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float
Base = declarative_base()

In [15]:
class brewery(Base):
    __tablename__ = "breweries" # required field  
    id = Column(Integer, primary_key=True, nullable=False)
    street_address = Column(String(100))
    zipcode = Column(Integer)
    austin_area = Column(String(100))
    untappd_website = Column(String(200))



class pop(Base):
    __tablename__ = "popular_beers" # required field  
    id = Column(Integer, primary_key=True, nullable=False)
    brewery_id = Column(Integer)
    name = Column(String(100))
    type = Column(String(100))
    abv = Column(Float)
    ibu = Column(Float)
    rating = Column(Float)
    rating_count= Column(Integer)


class rate(Base):
    __tablename__ = "top_rated_beers" # required field  
    id = Column(Integer, primary_key=True, nullable=False)
    brewery_id = Column(Integer)
    name = Column(String(100))
    type = Column(String(100))
    abv = Column(Float)
    ibu = Column(Float)
    rating = Column(Float)
    rating_count= Column(Integer)


In [16]:
breweries.to_sql(brewery.__tablename__, con, index=True, if_exists="append")
popular.to_sql(pop.__tablename__, con, index=True, if_exists="append")
rated.to_sql(rate.__tablename__, con, index=True, if_exists="append")

In [18]:
engine2 = create_engine("sqlite:///data/austin_breweries.sqlite")
con = engine2.connect()

In [19]:
session2 = Session(engine2)
# reflect an existing database into a new model
Bases = automap_base()
# reflect the tables
Bases.prepare(engine2, reflect=True)

# Save reference to the table
Bases.classes.keys()

[]